<!--NAVIGATION-->
<a href="https://colab.research.google.com/github/marcoteran/deeplearningmodule/blob/main/04_recurrentneuralnetworks/04_recurrentneuralnetworks.ipynb" target="_blank"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Abrir en Colab" title="Abrir y ejecutar en Google Colaboratory"></a>

## Ejemplos de código
# Deep Learning: Redes Neuronales Recurrentes

*Name:* Marco Teran
*E-mail:* marco.tulio.teran@gmail.com,
[Website](http://marcoteran.github.io/),
[Github](https://github.com/marcoteran),
[LinkedIn](https://www.linkedin.com/in/marcoteran/).

### Importar librerías importantes

Empezamos con las importaciones estándar:

In [1]:
import warnings
warnings.filterwarnings('ignore')
#import mlutils
import numpy as np
import pandas as pd
import importlib
import os
import time

# Visualizations
import matplotlib.pyplot as plt
#import seaborn as sns; sns.set()  # for plot styling
from IPython.display import Image
from IPython.core.display import HTML 
%matplotlib inline

#### Importar TensorFlow 2.0

In [2]:
# Deep learning
import tensorflow as tf
from tensorflow import keras
print(tf.__version__)

2.10.1


# ¿Qué son las Redes Neuronales Recurrentes?

Una **red neuronal recurrente** no tiene una estructura de capas definida, sino que permiten conexiones arbitrarias entre las neuronas, incluso pudiendo crear ciclos, con esto se consigue crear la temporalidad, permitiendo que la red tenga memoria.

Las redes neuronales recurrentes son muy potentes para todo lo que tiene que ver con el análisis secuencias, como puede ser el análisis de textos, sonido o video.

Existe multitud de tipos de redes neuronales dependiendo del número de capas ocultas y la forma de realizar la retropropagación, a continuación se detallarán las más conocidas y los usos de estas.
![Red Neuronal Recurrente](https://www.diegocalvo.es/wp-content/uploads/2017/07/red-neuronal-recurrente-768x251.png)

# Ejemplo: Programando una RNN generación de texto

Se mostrará cómo se programa una red neuronal recurrente (RNN) basándonos en un caso de estudio que trata de generar texto usando una RNN basada en caracteres. De esta manera, también podemos usar el caso de estudio para mostrar el uso de datos de texto.

En este ejemplo se entrena un modelo de red neuronal para predecir el siguiente carácter a partir de una secuencia de caracteres. Con este modelo intencionadamente simple, para mantener el carácter didáctico del ejemplo, se consiguen generar secuencias de texto más largas llamando al modelo repetidamente.

## Character-Level Language Models

Para intentar buscar un ejemplo lo más simple posible en el que podamos aplicar una red neuronal recurrente, hemos considerado usar el ejemplo de **Character level language model** propuesto por *Andrej Karpathy* en su artículo _«The Unreasonable Effectiveness of Recurrent Neural Networks210»_ (y parcialmente basado en su implementado en el tutorial Generate text with an RNN de la web de TensorFlow).

En realidad, se trata de uno de los modelos pioneros en procesado de texto a nivel de carácter, llamado *char-rnn*:
* Consiste en darle a la RNN una palabra; entonces se le pide que modele la distribución de probabilidad del siguiente carácter que le correspondería a la secuencia de caracteres anteriores.
* Con este modelo, si lo llamamos repetitivamente, podremos generar texto carácter a carácter. 

Como ejemplo, supongamos que solo tenemos un vocabulario de cuatro letras posibles ["a","h","l","o"], y queremos entrenar a una RNN en la secuencia de entrenamiento "hola".

Esta secuencia de entrenamiento es, de hecho, una fuente de 3 ejemplos de entrenamiento por separado:
* la probabilidad de "o" debería ser verosímil dada el contexto de "h"
* "l" debería ser verosímil en el contexto de "ho"
* "a" debería ser también verosímil dado el contexto de "hol”.

Para este propósito, como dataset usaremos la primera parte del libro **DEEP LEARNING Introducción práctica en Keras213** en texto plano. Se trata de un dataset muy pequeño de solo *30 000* palabras que, además, resulta en parte confuso al pasarlo a texto plano, pues ha resultado una mezcla de texto con código de programación. Pero incluso siendo un dataset extremadamente limitado para poder ser considerado un *corpus real*, nos sirve para generar como salida oraciones en las que —aunque no encontremos ni gramaticalmente ni semánticamente demasiado sentido— se puede apreciar que la estructura del texto de salida se asemeja a una frase real.

## Descarga y preprocesado de los datos

El primer paso en este ejemplo será descargar y preparar el conjunto de datos con el que entrenaremos nuestra red neuronal

In [3]:
# Subirlo en línea
#from google.colab import files
#se debe cargar el fichero “Libro-Deep-Learning-introduccion-practica-con-Keras-1a-parte.txt”
#files.upload()

# Subirlo manualmente
path_to_fileDL ='data/libro-deep-learning-introduccion-practica-con-keras-1a-parte.txt'

# Subirlo automaticamente
#path_to_fileDL = tf.keras.utils.get_file('Shakespear.txt', 'https://cs.stanford.edu/people/karpathy/char-rnn/shakespear.txt')
#path_to_fileDL = tf.keras.utils.get_file('libro-deep-learning-introduccion-practica-con-keras-1a-parte', 'https://github.com/marcoteran/deeplearningmodule/raw/main/04_recurrentneuralnetworks/data/libro-deep-learning-introduccion-practica-con-keras-1a-parte.txt')


In [4]:
text = open(path_to_fileDL, 'rb').read().decode(encoding='utf-8')

print('Longitud del texto: {} carácteres'.format(len(text)))

vocab = sorted(set(text))

print ('El texto está compuesto de estos {} carácteres:'.format(len(vocab)))
print (vocab)

Longitud del texto: 201709 carácteres
El texto está compuesto de estos 91 carácteres:
['\n', ' ', '!', '"', '#', '%', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\xad', 'ÿ', 'Š', '‡', '…']


Como estamos tratando el caso de estudio a nivel de carácter, podríamos considerar que aquí el corpus son los caracteres y, por tanto, sería un corpus muy pequeño. 

Recordemos que las redes neuronales solo procesan valores numéricos, no letras; por tanto, tenemos que traducir los caracteres a representación numérica. Para ello crearemos dos «tablas de traducción»: una de caracteres a números y otra de números a caracteres: 

In [5]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

Ahora tenemos un token con la representación de entero (integer) para cada carácter, como podemos ver ejecutando el siguiente código:

In [6]:
for char,_ in zip(char2idx, range(len(vocab))):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))

  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '"' :   3,
  '#' :   4,
  '%' :   5,
  "'" :   6,
  '(' :   7,
  ')' :   8,
  '*' :   9,
  '+' :  10,
  ',' :  11,
  '-' :  12,
  '.' :  13,
  '/' :  14,
  '0' :  15,
  '1' :  16,
  '2' :  17,
  '3' :  18,
  '4' :  19,
  '5' :  20,
  '6' :  21,
  '7' :  22,
  '8' :  23,
  '9' :  24,
  ':' :  25,
  ';' :  26,
  '<' :  27,
  '=' :  28,
  '>' :  29,
  '?' :  30,
  '@' :  31,
  'A' :  32,
  'B' :  33,
  'C' :  34,
  'D' :  35,
  'E' :  36,
  'F' :  37,
  'G' :  38,
  'H' :  39,
  'I' :  40,
  'J' :  41,
  'K' :  42,
  'L' :  43,
  'M' :  44,
  'N' :  45,
  'O' :  46,
  'P' :  47,
  'Q' :  48,
  'R' :  49,
  'S' :  50,
  'T' :  51,
  'U' :  52,
  'V' :  53,
  'W' :  54,
  'X' :  55,
  'Y' :  56,
  '[' :  57,
  ']' :  58,
  '_' :  59,
  'a' :  60,
  'b' :  61,
  'c' :  62,
  'd' :  63,
  'e' :  64,
  'f' :  65,
  'g' :  66,
  'h' :  67,
  'i' :  68,
  'j' :  69,
  'k' :  70,
  'l' :  71,
  'm' :  72,
  'n' :  73,
  'o' :  74,
  'p' :  75,
  'q' :  76,

Y con esta función, inversa a la anterior, podemos pasar el texto (todo el libro) a enteros: 

In [7]:
text_as_int = np.array([char2idx[c] for c in text])

Para comprobarlo podemos mostrar los 50 primeros caracteres del texto contenido en el tensor text_as_int: 

In [8]:
print ('texto: {}'.format(repr(text[:50])))
print ('{}'.format(repr(text_as_int[:50])))

texto: 'Prologo\nEn 1953, Isaac Asimov publico Segunda Fund'
array([47, 77, 74, 71, 74, 66, 74,  0, 36, 73,  1, 16, 24, 20, 18, 11,  1,
       40, 78, 60, 60, 62,  1, 32, 78, 68, 72, 74, 81,  1, 75, 80, 61, 71,
       68, 62, 74,  1, 50, 64, 66, 80, 73, 63, 60,  1, 37, 80, 73, 63])


### Preparación de los datos para ser usados por la RNN

Para entrenar el modelo, prepararemos unas secuencias de caracteres como entradas y salida de un tamaño determinado.
* En nuestro ejemplo, hemos definido el tamaño de **100 caracteres** con la variable ``seq_length``
* Empezamos dividiendo el texto que tenemos en secuencias de ``seq_length+1`` de caracteres
* Luego construiremos los datos de entrenamiento compuestos por las entradas de ``seq_length`` caracteres y las salidas correspondientes que contienen la misma longitud de texto (excepto que se desplaza un carácter a la derecha).
* Ejemplo: “Hola”. Suponiendo un ``seq_length=3``, la secuencia de entrada será ``"Hol"``, y la de salida será ``"ola"``

Usaremos la función ``tf.data.Dataset.from_tensor_slices``, que crea un conjunto de datos con el contenido del ``tensortext_as_int`` que contiene el texto, al que podremos aplicar el método ``batch()`` para dividir este conjunto de datos en secuencias de seq_length+1 de índice de caracteres:

In [9]:
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

seq_length = 100
 
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

Podemos comprobar que sequences contiene el texto dividido en paquetes de 101 caracteres como esperamos (por ejemplo, mostremos las 10 primeras secuencias):

In [10]:
for item in sequences.take(10):
  print(repr(''.join(idx2char[item.numpy()])))

'Prologo\nEn 1953, Isaac Asimov publico Segunda Fundacion, el tercer libro de la saga de la Fundacion ('
'o el decimotercero segun otras fuentes, este es un tema de debate). En Segunda Fundacion aparece por '
'primera vez Arkady Darell, uno de los principales personajes de la parte final de la saga. En su prim'
'era escena, Arkady, que tiene 14 anos, esta haciendo sus tareas escolares. En concreto, una redaccion'
' que lleva por titulo ?El Futuro del Plan Sheldon?. Para hacer la redaccion, Arkady esta utilizando u'
'n ?transcriptor?,un dispositivo que convierte su voz en palabras escritas. Este tipo de dispositivo, '
'que para Isaac Asimov era ciencia ficcion en 1953, lo tenemos al alcance de la mano en la mayoria de '
'nuestros smartphones, y el Deep Learning es uno de los responsables de que ya tengamos este tipo de a'
'plicaciones, siendo la tecnologia otro de ellos.En la actualidad disponemos de GPUs (Graphics Process'
'or Units), que solo cuestan alrededor de 100 euros, que estari

De esta secuencia se obtiene el conjunto de datos de entrenamiento que contiene tanto los datos de entrada (desde la posición 0 a la 99) como los datos de salida (desde la posición 1 a la 100).

Para ello, se crea una función que realiza esta tarea y se aplica a todas las secuencias usando el método ``map()`` de la siguiente forma:

In [11]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

In [12]:
dataset = sequences.map(split_input_target)

En este punto, dataset contiene un conjunto de parejas de secuencias de texto (con la representación numérica de los caracteres) donde el primer componente de la pareja contiene un paquete con una secuencia de 100 caracteres del texto original, y el segundo contiene su correspondiente salida (etiqueta), también de 100 caracteres. 

Podemos comprobarlo visualizándolo por pantalla (por ejemplo mostrando la primera pareja): 

In [13]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'Prologo\nEn 1953, Isaac Asimov publico Segunda Fundacion, el tercer libro de la saga de la Fundacion '
Target data: 'rologo\nEn 1953, Isaac Asimov publico Segunda Fundacion, el tercer libro de la saga de la Fundacion ('


En este punto del código disponemos de los datos de entrenamiento en el tensor ``dataset`` en forma de parejas de secuencias de 100 integers de 64 bits que representan un carácter del vocabulario:

In [14]:
print (dataset)

<MapDataset element_spec=(TensorSpec(shape=(100,), dtype=tf.int32, name=None), TensorSpec(shape=(100,), dtype=tf.int32, name=None))>


* Los datos ya están preprocesados en el formato que se requiere para ser usados en el entreno de la red neuronal
* **Recordar:** en redes neuronales los datos se agrupan en *batches* antes de pasarlos al modelo.
    * En nuestro caso hemos decidido un *tamaño de batch* de 64, que nos facilita la explicación pero. Este es un hiperparámetro importante, y para ajustarlo correctamente hay que tener en cuenta diferentes factores, como el tamaño de la memoria disponible
    * Para crear los batches de parejas de secuencias hemos considerado usar ``tf.data`` que, además, nos permite *barajar* las secuencias previamente

In [15]:
BATCH_SIZE = 64

BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

print (dataset)

<BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int32, name=None), TensorSpec(shape=(64, 100), dtype=tf.int32, name=None))>


En el tensor ``dataset`` disponemos de los datos de entrenamiento ya listos para ser usados para ajustar los parámetros del modelo con el proceso de entrenamiento:
* *batches* compuestos de 64 parejas de secuencias de 100 integers de 64 bits que representan el carácter correspondiente en el vocabulario.

### Construcción del modelo RNN

Para construir el modelo usaremos la API secuencial de Keras ``tf.keras.Sequential``.
Usaremos una versión mínima de RNN para facilitar la explicación, que contenga solo una capa LSTM.

In [16]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

En concreto, definimos una red de solo 3 capas:

In [17]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = Sequential()
    model.add(Embedding(input_dim=vocab_size, 
                        output_dim=embedding_dim,
                        batch_input_shape=[batch_size, None]))
    model.add(LSTM(rnn_units,
                   return_sequences=True,
                   stateful=True,
                   recurrent_initializer='glorot_uniform'))
    model.add(Dense(vocab_size))
    return model

1. La primera capa es de tipo ``word embedding``, como las que antes hemos presentado muy brevemente, que mapea cada carácter de entrada en un vector embedding. Esta ``capa tf.keras.layers.Embedding`` permite especificar varios argumentos que se pueden consultar en todo detalle en el manual de TensorFlow.
    * En nuestro caso, el primero que especificamos es el tamaño del vocabulario, indicado con el argumento ``vocab_size``, que indica cuántos vectores embedding tendrá la capa.
    * A continuación, indicamos las dimensiones de estos vectores embedding mediante el argumento ``embedding_dim``, que en nuestro caso hemos decidido que sea 256.
    * Finalmente, se indica el tamaño del batch que usaremos para entrenar, en nuestro caso 64.

2. La segunda capa es de tipo LSTM. Esta capa ``tf.keras.layers.LSTM`` tiene varios argumentos posibles que se pueden consultar en el manual de TensorFlow; aquí solo usaremos algunos y dejaremos los valores por defecto del resto.
    * Quizás el más importante sea el número de neuronas recurrentes, que se indica con el argumento ``units`` y que en nuestro caso hemos decidido que sea 1024 neuronas.
    * Con ``return_sequence`` se indica que queremos predecir el carácter siguiente a todos los caracteres de entrada, no solo el siguiente al último carácter.
    * El argumento ``stateful`` indica el uso de las capacidades de memoria de la red entre batches. Si este argumento está instanciado a **``False``** se indica que a cada nuevo batch se inicializan las memory cells, mientras que si está a **``True``** se está indicando que para cada batch se mantendrán las actualizaciones hechas durante la ejecución del batch anterior.
    * El último argumento que usamos es ``recurrent_kernel``, donde indicamos cómo se deben inicializar los pesos de las matrices internas de la red. En este caso usamos la distribución uniforme ``glorot_uniform``, habitual en estos casos.

3. Finalmente la última capa es de tipo ``Dense``. Aquí es importante el argumento ``units`` que nos dice cuántas neuronas tendrá la capa y que nos marcará la dimensión de la salida. En nuestro caso será igual al tamaño de nuestro vocabulario ``(vocab_size)``

In [18]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

Es interesante usar el método summary() para visualizar la estructura del modelo:

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           23296     
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 dense (Dense)               (64, None, 91)            93275     
                                                                 
Total params: 5,363,547
Trainable params: 5,363,547
Non-trainable params: 0
_________________________________________________________________


* Podemos comprobar que la capa LSTM consta de muchos parámetros (más de 5 millones) como era de esperar.
* Para cada carácter de entrada (transformado a su equivalente numérico), el modelo busca su vector de embedding correspondiente, y luego ejecuta la capa LSTM con este vector embedding como entrada.
* A la salida de la LSTM aplica la capa Dense para decidir cuál es el siguiente carácter.

Inspeccionemos las dimensiones de los tensores para poder comprender más a fondo el modelo. Fijémonos en el primer batch del conjunto de datos de entrenamiento y observemos su forma:

In [20]:
for input_example_batch, target_example_batch in dataset.take(1):
  print("Input:", input_example_batch.shape, "# (batch_size, sequence_length)")
  print("Target:", target_example_batch.shape, "# (batch_size, sequence_length)")


Input: (64, 100) # (batch_size, sequence_length)
Target: (64, 100) # (batch_size, sequence_length)


Vemos que en esta red la secuencia de entrada son batches de 100 caracteres, pero el modelo una vez entrenado puede ser ejecutado con cualquier tamaño de cadena de entrada.

Como salida, el modelo nos devuelve un tensor con una dimensión adicional con la verosimilitud para cada carácter del vocabulario

In [21]:
for input_example_batch, target_example_batch in dataset.take(1):  
    example_batch_predictions = model(input_example_batch)
    print("Prediction: ", example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

Prediction:  (64, 100, 91) # (batch_size, sequence_length, vocab_size)


El siguiente paso es elegir uno de los caracteres. No se eligirá el carácter más *«probable»* (mediante ``argmax``), puesto que el modelo puede entrar en un bucle.

Lo que se hará es obtener una muestra de la distribución de salida. Prueba para el primer ejemplo en el batch:

In [22]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices_characters = tf.squeeze(sampled_indices,axis=-1).numpy()

In [23]:
sampled_indices_characters

array([ 8, 16, 42, 38, 23,  5, 74, 85, 54, 37,  0,  7, 77, 50, 70, 81, 18,
       34, 29, 70, 58, 44, 52, 40, 65,  6, 46, 79, 36,  3,  3, 27, 29, 62,
        7, 23,  6,  8, 78, 88, 25,  2, 31,  6, 45, 65, 64, 37, 72, 21, 49,
       71, 16, 71, 87,  7, 61,  5, 29, 75, 24, 16, 82, 61, 89, 17, 14, 59,
       32, 70, 83, 79, 11, 58, 37, 78, 90, 58, 47, 16, 69,  8, 29, 19, 83,
       27, 18,  6, 41, 29, 42, 20, 50, 61, 35, 33, 74, 88, 10,  1],
      dtype=int64)

Con ``tf.random.categorical`` se obtiene una muestra de una distribución categórica y con ``squeeze`` se eliminan la dimensiones del tensor de tamaño 1.
Así, en cada instante de tiempo se obtiene una predicción del índice del siguiente carácter

## Entrenar del modelo RNN

En este punto, el problema puede tratarse como un problema de clasificación estándar para el que debemos definir la función de pérdida y el optimizador

Para la función de pérdida usaremos la función estándar ``tf.keras.losses.sparse_categorical_crossentropy``, puesto que estamos considerando datos categóricos. Dado que el retorno hemos visto que se trata de unos valores de verosimilitud (no de probabilidades —como si hubiéramos ya aplicado softmax—) se instanciará el argumento ``from_logits`` a **``True``**

In [24]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

Para el optimizador, usaremos ``tf.keras.optimizers.Adam`` con los argumentos por defecto del optimizador Adam. Con la función de lossdefinida, y usando el optimizador Adam con sus argumentos por defecto, ya podemos llamar al método ``compile()`` de la siguiente manera:

In [25]:
model.compile(optimizer='adam', loss=loss)

#### Configurar el *checkpoints*

En este ejemplo aprovecharemos para usar los *Checkpoints*:
* Una técnica de tolerancia de fallos para procesos cuyo tiempo de ejecución es muy largo.
* La idea es guardar una instantánea del estado del sistema *periódicamente* para recuperar desde ese punto la ejecución en caso de fallo del sistema.

Cuando entrenamos modelos Deep Learning, el *Checkpoint* lo forman básicamente los pesos del modelo:
* Estos Checkpoint se pueden usar también para hacer predicciones

La librería de Keras proporciona *Checkpoints* a través de la API ``Callbacks``. Concretamente usaremos ``tf.keras.callbacks.ModelCheckpoint`` para especificar cómo salvar los *Checkpoints* a cada *epoch* durante el entrenamiento, a través de un argumento en el método ``fit()`` del modelo.
* En el código debemos especificar el directorio en el que se guardarán los *Checkpoints* que salvaremos y el nombre del fichero (al que añadiremos el número de *epoch* para nuestra comodidad): 

In [26]:
 # directorio
checkpoint_dir = './training_checkpoints'

# nombre fichero
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

## Training

Ahora ya está todo preparado para empezar a entrenar la red con el método ``fit()``: 

In [27]:
EPOCHS=50
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/50
31/31 [==============================] - 3s 66ms/step - loss: 3.2184
Epoch 2/50
31/31 [==============================] - 2s 65ms/step - loss: 2.7690
Epoch 3/50
31/31 [==============================] - 2s 62ms/step - loss: 2.4425
Epoch 4/50
31/31 [==============================] - 2s 65ms/step - loss: 2.2399
Epoch 5/50
31/31 [==============================] - 2s 62ms/step - loss: 2.1310
Epoch 6/50
31/31 [==============================] - 2s 63ms/step - loss: 2.0236
Epoch 7/50
31/31 [==============================] - 2s 63ms/step - loss: 1.9147
Epoch 8/50
31/31 [==============================] - 2s 63ms/step - loss: 1.8025
Epoch 9/50
31/31 [==============================] - 2s 63ms/step - loss: 1.6970
Epoch 10/50
31/31 [==============================] - 2s 64ms/step - loss: 1.5959
Epoch 11/50
31/31 [==============================] - 2s 66ms/step - loss: 1.5032
Epoch 12/50
31/31 [==============================] - 2s 64ms/step - loss: 1.4219
Epoch 13/50
31/31 [==================

Se puede inspeccionar el contenido del directorio training_checkpoints para comprobar que se han generado estos ficheros mediante el comando:

In [28]:
!ls training_checkpoints

"ls" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


## Generación de texto usando el modelo RNN

Ahora que ya tenemos entrenado el modelo, pasemos a usarlo para generar texto.
Para mantener este paso de predicción simple, vamos a usar un tamaño de *batch* de 1.
Debido a la forma en que se pasa el estado de la RNN de un instante de tiempo al siguiente, el modelo solo acepta un tamaño de batch fijo una vez construido.
Por ello, para poder ejecutar el modelo con un tamaño de *batch* diferente, necesitamos reconstruir manualmente el modelo con el método ``build()`` y restaurar sus pesos desde el *Checkpoint* (cogemos el ultimo con ``tf.train.latest_checkpoint()``)

In [29]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints\\ckpt_50'

In [30]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

Ahora que tenemos el modelo entrenado y preparado para usar, generaremos texto a partir de una palabra de partida con el siguiente código.

El código empieza con inicializaciones como:
* Definir el número de caracteres a predecir con la variable ``num_generate``
* Convertir la palabra inicial (``start_string``) a su correspondiente representación numérica y preparar lo tensores necesarios
```Python
num_generate = 500
    input_eval = [char2idx[s] for s in start_string]

    input_eval = tf.expand_dims(input_eval, 0)
    text_generated = []
```
Usando la misma idea del código original **char-rnn** de *Andrey Karpathy*, se usa una variable ``temperature`` para decidir cuán conservador en sus predicciones queremos que sea nuestro modelo. En nuestro ejemplo la hemos inicializado a 0.5
* Con *«temperaturas altas»* (hasta 1) se permitirá más creatividad al modelo para generar texto, pero a costa de más errores (por ejemplo, errores ortográficos, etc.).
* Con *«temperaturas bajas»* habrá menos errores pero el modelo mostrará poca creatividad.
Es posible revisar diferentes valores y ver su efecto
```Python
temperature = 0.5``
```
A partir de este momento empieza el bucle para generar los caracteres que le hemos indicado (que use el carácter de entrada la primera vez) y luego sus propias predicciones como entrada a cada nueva iteración al modelo RNN:
```Python
 model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
```
Recordemos que estamos en un batch de tamaño 1 pero el modelo retorna el tensor del batch con las dimensiones con que lo habíamos entrenado y, por tanto, debemos reducir la dimensión batch:
```Python
predictions = tf.squeeze(predictions, 0)
```
Luego, se usa una distribución categórica para calcular el índice del carácter predicho: 
```Python
predictions = predictions / temperature
predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
```
Este carácter acabado de predecir se usa como nuestra próxima entrada al modelo, retroalimentando el modelo para que ahora tenga más contexto (en lugar de una sola letra).

Después de predecir la siguiente letra, se retroalimenta nuevamente, y así sucesivamente, de manera que aprende a medida que se obtiene más contexto de los caracteres predichos previamente:
```Python
input_eval = tf.expand_dims([predicted_id], 0)
text_generated.append(idx2char[predicted_id])
```

In [31]:
def generate_text(model, start_string):
    
    num_generate = 500
    input_eval = [char2idx[s] for s in start_string]

    input_eval = tf.expand_dims(input_eval, 0)
    text_generated = []


    temperature = 0.5

    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
      
        predictions = tf.squeeze(predictions, 0)

        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()


        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

Ahora que se ha descrito cómo se ha programado la función ``generate_txt`` probemos cómo se comporta el modelo.
Empecemos con una palabra que no conoce el corpus, por ejemplo **«Domingo»**, que nada tiene que ver con *Deep Learning*, aunque es a quien hemos dedicado este libro

In [32]:
print(generate_text(model, start_string=u"Domingo"))

Domingonde, se proponemos una con distinta capa convolucional es la que mostramos en la siguiente figura, donde se aplica el torcando con el tecnologia de computacion y estan abriendo el espectro de que cosas puede hacer un conocimiento solido al lector. Esto conlleva que en sus paginas se iran intercalando conocimiento de voz, reconocimiento de objetos visuales, deteccion de objetos y muchos otros dominios, y cada parametro e hiperparametro importa mucho en el resultado final; esto lo hace muchos anos


Probemos ahora con una palabra como «Activación» o «Redes», a ver qué pasa: 

In [33]:
print(generate_text(model, start_string=u"Activacion"))

Activacion al final de cada epecho en la comunidad de vision por computador tambien son enormes: ahora nuestros ordenadores, por ejemplo, podemos considerar usar la imagen de 5?5 se ha convertido en un filtro de tamano mas reducido de 2?2. Pero en realidad los pasos a seguir para ejecutar un Jupyter en Cloud sin el que todas las neuronas de este primer nivel de neuronas ocultas, como en el caso de las redes neuronales y tiene tensores dinamicos, entre ellos se dedica mas tiempo a ?teoria? y en algun otro 


In [34]:
print(generate_text(model, start_string=u"Redes"))

Redes@QEG#RRVKKPRKKKKKKKKKVKKKKMKLKKGKVKGKKKKKKKKKKKKKKKKKKKKKKKKKKKGKKKKKKKKKKKKKKKKKKKKKKKKSKKKKKKKKKKKKKKKKKKKKWKK‡EKKKKKKKKKKKKKKKKGKKRKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKwKKGKKVGKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK‡EQKKRE…KKKMKKKKGKKKKKKKKKKKzwKKKKKKQKKKKKKMKKKKVKKKKKKKKKEKGKKKKKKKKQK#KKKKKKKKKGKKKKKKKKKKKKKKKKK#KKKKKKKKKKKKKKKKKKKKKKKK*MGYGKKMGKKKKK*KKKKKKKKKKKKxK‡RKKKKKKwKKKKKKKKKKKEKKKKKKKKKKKKKMGKKKKKGKKKKGKKKKKKKKKKVKKEKKK*KKKKKRKKKKKKKKKJKKKKEKKKKKKKKKKKKKKKKKKKK*wKQKKVKEKGKKKKKGKKKKKKKKKKKKKKK
